In [2]:
import pandas as pd
import numpy as np

class MarketAnalyzer:
    def __init__(self, timeframe):
        self.timeframe = timeframe

    def resample_data(self, data):
        data = data.copy()
        data['datetime'] = pd.to_datetime(data.iloc[:, 0])
        data.set_index('datetime', inplace=True)
        resampled = data.resample(self.timeframe).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()
        return resampled

    def calculate_indicators(self, data):
        if len(data) < 200:
            raise ValueError(f"Insufficient data points after resampling. Need at least 200, but got {len(data)}.")

        close = data['close']
        high = data['high']
        low = data['low']
        volume = data['volume']

        ma20 = close.rolling(window=20).mean()
        ma200 = close.rolling(window=200).mean()

        atr = self.calculate_atr(high, low, close)

        highest_high = high.rolling(window=100).max()
        lowest_low = low.rolling(window=100).min()
        
        premium_zone = highest_high - atr
        discount_zone = lowest_low + atr
        equilibrium = (premium_zone + discount_zone) / 2

        return {
            'ma20': ma20,
            'ma200': ma200,
            'atr': atr,
            'premium_zone': premium_zone,
            'discount_zone': discount_zone,
            'equilibrium': equilibrium,
            'volume': volume
        }

    def calculate_atr(self, high, low, close, period=14):
        tr = np.maximum(high - low, 
                        np.abs(high - close.shift(1)),
                        np.abs(low - close.shift(1)))
        return tr.rolling(window=period).mean()

    def calculate_volume_profile(self, data, indicators, lookback=200):
        """Optimized volume profile calculation"""
        if len(data) < lookback:
            raise ValueError(f"Insufficient data points for volume profile. Need at least {lookback}, but got {len(data)}.")
            
        recent_data = data.iloc[-lookback:]
        price_range = recent_data['high'].max() - recent_data['low'].min()
        num_rows = 25
        
        # Pre-calculate bins
        bins = np.linspace(recent_data['low'].min(), recent_data['high'].max(), num_rows + 1)
        
        # Use numpy histogram for faster calculation
        volume_profile, _ = np.histogram(recent_data['close'], 
                                       bins=bins, 
                                       weights=recent_data['volume'])
        
        return volume_profile

    def detect_fair_value_gaps(self, data):
        bullish_fvg = (data['low'] > data['high'].shift(2)) & (data['close'].shift(1) > data['open'].shift(1))
        bearish_fvg = (data['high'] < data['low'].shift(2)) & (data['close'].shift(1) < data['open'].shift(1))
        
        return bullish_fvg, bearish_fvg

    def detect_order_blocks(self, data, block_size=10, block_threshold=0.05):
        heights = np.abs(data['high'] - data['low'])
        height_sum = heights.rolling(window=block_size).sum()
        block_height = height_sum / block_size
        block_thresholds = block_threshold * block_height

        order_blocks = pd.Series(False, index=data.index)
        for i in range(block_size, len(data)):
            if (data['high'].iloc[i-1] < data['high'].iloc[i] and 
                data['low'].iloc[i-1] < data['low'].iloc[i] and
                heights.iloc[i] < block_thresholds.iloc[i]):
                order_blocks.iloc[i] = True

        return order_blocks

    def identify_market_state(self, data, indicators):
        if len(data) < 200:
            return 'Insufficient Data', {}

        current_price = data['close'].iloc[-1]
        ma20 = indicators['ma20'].iloc[-1]
        ma200 = indicators['ma200'].iloc[-1]
        premium_zone = indicators['premium_zone'].iloc[-1]
        discount_zone = indicators['discount_zone'].iloc[-1]
        equilibrium = indicators['equilibrium'].iloc[-1]

        ma_diff = (ma20 - ma200) / ma200
        
        volume_profile = self.calculate_volume_profile(data, indicators)
        high_volume_level = np.percentile(volume_profile, 80)
        low_volume_level = np.percentile(volume_profile, 20)

        bullish_fvg, bearish_fvg = self.detect_fair_value_gaps(data)
        order_blocks = self.detect_order_blocks(data)

        if current_price > premium_zone:
            market_state = 'Premium'
        elif current_price < discount_zone:
            market_state = 'Discount'
        elif abs(ma_diff) < 0.01:
            if np.max(volume_profile) > high_volume_level:
                market_state = 'Narrow - High Volume'
            elif np.max(volume_profile) < low_volume_level:
                market_state = 'Narrow - Low Volume'
            else:
                market_state = 'Narrow'
        elif ma_diff > 0.01:
            if ma20 > indicators['ma20'].iloc[-2] and ma200 > indicators['ma200'].iloc[-2]:
                market_state = 'Trending Up'
            else:
                market_state = 'Wide - Bullish'
        elif ma_diff < -0.01:
            if ma20 < indicators['ma20'].iloc[-2] and ma200 < indicators['ma200'].iloc[-2]:
                market_state = 'Trending Down'
            else:
                market_state = 'Wide - Bearish'
        else:
            market_state = 'Equilibrium'

        levels = {
            'support': data['low'].tail(20).min(),
            'resistance': data['high'].tail(20).max(),
            'current_price': current_price,
            'equilibrium': equilibrium,
            'bullish_fvg': bullish_fvg.iloc[-1],
            'bearish_fvg': bearish_fvg.iloc[-1],
            'order_block': order_blocks.iloc[-1]
        }

        return market_state, levels

    def generate_trading_signals(self, data, indicators):
        """Optimized signal generation while maintaining original logic"""
        signals = pd.DataFrame(index=data.index, columns=['signal', 'strategy', 'entry_price', 'market_state'])
        
        # Pre-calculate market states for all data points
        market_state, levels = self.identify_market_state(data, indicators)
        
        # Pre-calculate moving averages
        current_prices = data['close']
        ma20 = indicators['ma20']
        
        last_signal = None
        signal_cooldown = 0
        
        # Use vectorized operations where possible
        highs = data['high']
        lows = data['low']
        
        for i in range(200, len(data)):
            if signal_cooldown > 0:
                signal_cooldown -= 1
                continue
                
            current_price = current_prices.iloc[i]
            ma20_current = ma20.iloc[i]
            
            signal = None
            strategy = None
            
            if market_state == 'Premium':
                signal = -1
                strategy = "Sell in Premium Zone"
            elif market_state == 'Discount':
                signal = 1
                strategy = "Buy in Discount Zone"
            elif 'Narrow' in market_state:
                if current_price > highs.iloc[i-1]:
                    signal = 1
                    strategy = "Breakout - Buy"
                elif current_price < lows.iloc[i-1]:
                    signal = -1
                    strategy = "Breakout - Sell"
            elif market_state == 'Trending Up':
                if current_price > ma20_current and lows.iloc[i] < ma20_current:
                    signal = 1
                    strategy = "Buy the Dip"
            elif market_state == 'Trending Down':
                if current_price < ma20_current and highs.iloc[i] > ma20_current:
                    signal = -1
                    strategy = "Sell the Rally"
            elif 'Wide' in market_state:
                if 'Bullish' in market_state and current_price < ma20_current:
                    signal = 1
                    strategy = "Buy the Dip in Wide Market"
                elif 'Bearish' in market_state and current_price > ma20_current:
                    signal = -1
                    strategy = "Sell the Rally in Wide Market"
            
            if signal is not None and signal != last_signal:
                signals.loc[signals.index[i]] = [signal, strategy, current_price, market_state]
                last_signal = signal
                signal_cooldown = 5
        
        return signals.dropna()
    
    def _calculate_ma(self, data, window):
        """Optimized moving average calculation"""
        weights = np.ones(window) / window
        return np.convolve(data, weights, mode='valid')

    def _calculate_rolling_max(self, data, window):
        """Optimized rolling maximum calculation"""
        return pd.Series(data).rolling(window=window).max()

    def _calculate_rolling_min(self, data, window):
        """Optimized rolling minimum calculation"""
        return pd.Series(data).rolling(window=window).min()

    def _calculate_atr_optimized(self, high, low, close, period=14):
        """Optimized ATR calculation"""
        tr1 = high - low
        tr2 = np.abs(high - np.roll(close, 1))
        tr3 = np.abs(low - np.roll(close, 1))
        tr = np.maximum.reduce([tr1, tr2, tr3])
        atr = pd.Series(tr).rolling(window=period).mean()
        return atr

    def analyze_market(self, data):
        resampled_data = self.resample_data(data)
        
        if len(resampled_data) < 200:
            return {
                'resampled_data': resampled_data,
                'market_state': 'Insufficient Data',
                'indicators': None,
                'signals': None,
                'levels': None
            }

        indicators = self.calculate_indicators(resampled_data)
        market_state, levels = self.identify_market_state(resampled_data, indicators)
        signals = self.generate_trading_signals(resampled_data, indicators)

        return {
            'resampled_data': resampled_data,
            'market_state': market_state,
            'indicators': indicators,
            'signals': signals,
            'levels': levels
        }
    # Add these new methods to your MarketAnalyzer class

    def analyze_order_flow(self, data, indicators):
        """Analyzes order flow patterns"""
        delta = pd.DataFrame(index=data.index)
        
        # Calculate buying/selling pressure
        delta['buying_pressure'] = (data['close'] - data['low']) / (data['high'] - data['low'])
        delta['selling_pressure'] = 1 - delta['buying_pressure']
        
        # Calculate volume delta
        delta['volume_delta'] = np.where(data['close'] > data['open'], 
                                       data['volume'], 
                                       -data['volume'])
        
        # Calculate cumulative delta
        delta['cumulative_delta'] = delta['volume_delta'].cumsum()
        
        # Identify absorption levels (high volume with little price movement)
        price_movement = (data['high'] - data['low']) / data['low']
        volume_norm = data['volume'] / data['volume'].rolling(20).mean()
        delta['absorption'] = (volume_norm > 1.5) & (price_movement < 0.002)
        
        return delta

    def analyze_vpa(self, data, window=20):
        """Volume Price Analysis"""
        vpa = pd.DataFrame(index=data.index)
        
        # Calculate relative volume
        vpa['relative_volume'] = data['volume'] / data['volume'].rolling(window).mean()
        
        # Identify high volume bars
        vpa['high_volume'] = vpa['relative_volume'] > 1.5
        
        # Price spread analysis
        bar_size = (data['high'] - data['low']) / data['low']
        vpa['wide_range'] = bar_size > bar_size.rolling(window).mean() + bar_size.rolling(window).std()
        
        # Volume climax detection
        vpa['volume_climax'] = (vpa['high_volume'] & 
                               vpa['wide_range'] & 
                               (data['close'] == data['high']) | 
                               (data['close'] == data['low']))
        
        # Calculate price rejection
        upper_wick = data['high'] - data[['open', 'close']].max(axis=1)
        lower_wick = data[['open', 'close']].min(axis=1) - data['low']
        body = abs(data['close'] - data['open'])
        
        vpa['upper_rejection'] = (upper_wick > body) & vpa['high_volume']
        vpa['lower_rejection'] = (lower_wick > body) & vpa['high_volume']
        
        return vpa

    def analyze_market_structure(self, data, indicators):
        """Enhanced market structure analysis"""
        structure = pd.DataFrame(index=data.index)
        
        # Identify swing points
        window = 5
        structure['swing_high'] = data['high'].rolling(window=window, center=True).max() == data['high']
        structure['swing_low'] = data['low'].rolling(window=window, center=True).min() == data['low']
        
        # Higher highs and lower lows
        highs = data[structure['swing_high']]['high']
        lows = data[structure['swing_low']]['low']
        
        structure['higher_high'] = False
        structure['lower_low'] = False
        
        for i in range(len(highs)):
            if i > 0:
                structure.loc[highs.index[i], 'higher_high'] = highs.iloc[i] > highs.iloc[i-1]
                
        for i in range(len(lows)):
            if i > 0:
                structure.loc[lows.index[i], 'lower_low'] = lows.iloc[i] < lows.iloc[i-1]
        
        # Identify consolidation zones
        volatility = data['close'].pct_change().rolling(window=20).std()
        structure['consolidation'] = volatility < volatility.rolling(window=100).mean() * 0.5
        
        return structure

    def analyze_volatility(self, data, window=20):
        """Advanced volatility analysis"""
        volatility = pd.DataFrame(index=data.index)
        
        # Calculate different volatility metrics
        returns = data['close'].pct_change()
        volatility['historical'] = returns.rolling(window=window).std() * np.sqrt(252)
        
        # Parkinson volatility (uses high-low range)
        hl_range = np.log(data['high'] / data['low'])
        volatility['parkinson'] = np.sqrt(1 / (4 * np.log(2)) * hl_range.rolling(window=window).mean() * 252)
        
        # Identify volatility regimes
        vol_percentile = volatility['historical'].rolling(window=100).apply(
            lambda x: pd.Series(x).rank(pct=True).iloc[-1]
        )
        
        volatility['regime'] = pd.cut(vol_percentile, 
                                    bins=[0, 0.25, 0.75, 1], 
                                    labels=['low', 'normal', 'high'])
        
        # Volatility breakout detection
        volatility['breakout'] = volatility['historical'] > volatility['historical'].rolling(window=100).max()
        
        return volatility

    def enhance_market_state(self, data, indicators):
        """Enhanced market state identification incorporating new analyses"""
        # Get base market state
        base_state, levels = self.identify_market_state(data, indicators)
        
        # Get additional analyses
        order_flow = self.analyze_order_flow(data, indicators)
        vpa = self.analyze_vpa(data)
        structure = self.analyze_market_structure(data, indicators)
        volatility = self.analyze_volatility(data)
        
        # Enhance market state based on additional factors
        enhanced_state = base_state
        
        # Check for absorption
        if order_flow['absorption'].iloc[-1]:
            enhanced_state += " - Absorption"
        
        # Check for climax
        if vpa['volume_climax'].iloc[-1]:
            enhanced_state += " - Climax"
        
        # Check structure
        if structure['consolidation'].iloc[-1]:
            enhanced_state += " - Consolidating"
        
        # Add volatility context
        enhanced_state += f" ({volatility['regime'].iloc[-1]} volatility)"
        
        # Enhanced levels
        enhanced_levels = levels.copy()
        enhanced_levels.update({
            'cumulative_delta': order_flow['cumulative_delta'].iloc[-1],
            'absorption_level': order_flow['absorption'].iloc[-1],
            'volume_climax': vpa['volume_climax'].iloc[-1],
            'volatility_percentile': volatility['historical'].iloc[-1],
            'in_consolidation': structure['consolidation'].iloc[-1]
        })
        
        return enhanced_state, enhanced_levels



def analyze_crude_oil(historical_data, timeframe):
    analyzer = MarketAnalyzer(timeframe)
    analysis = analyzer.analyze_market(historical_data)
    resampled_data = analysis['resampled_data']
    current_price = analysis['levels']['current_price']
    atr = analysis['indicators']['atr'].iloc[-1] if analysis['indicators'] else 0

    print("Market Analysis:")
    print(f"Current Market State: {analysis['market_state']}")
    
    # Vertical spread strategy matrix
    market_state_strategies = {
        'Premium': {
            'strategy': "Bear Call Spread",
            'direction': "Bearish",
            'strikes': lambda: (round(current_price, 2), 
                              round(current_price + atr, 2)),
            'max_profit': "Net premium received",
            'max_loss': "Difference between strikes - premium received",
            'breakeven': "Short strike + net premium",
            'greek_profile': "Negative delta, positive theta, negative vega"
        },
        'Discount': {
            'strategy': "Bull Put Spread",
            'direction': "Bullish",
            'strikes': lambda: (round(current_price - atr, 2), 
                              round(current_price, 2)),
            'max_profit': "Net premium received",
            'max_loss': "Difference between strikes - premium received",
            'breakeven': "Short strike - net premium",
            'greek_profile': "Positive delta, positive theta, negative vega"
        },
        'Trending Up': {
            'strategy': "Bull Call Spread",
            'direction': "Bullish",
            'strikes': lambda: (round(current_price, 2), 
                              round(current_price + 2*atr, 2)),
            'max_profit': "Difference between strikes - net debit",
            'max_loss': "Net premium paid",
            'breakeven': "Long strike + net debit",
            'greek_profile': "Positive delta, negative theta, positive vega"
        },
        'Trending Down': {
            'strategy': "Bear Put Spread",
            'direction': "Bearish",
            'strikes': lambda: (round(current_price - 2*atr, 2), 
                              round(current_price, 2)),
            'max_profit': "Difference between strikes - net debit",
            'max_loss': "Net premium paid",
            'breakeven': "Long strike - net debit",
            'greek_profile': "Negative delta, negative theta, positive vega"
        },
        'Narrow - High Volume': {
            'strategy': "Iron Condor",
            'direction': "Neutral",
            'strikes': lambda: (round(current_price - 1.5*atr, 2),
                              round(current_price - 0.5*atr, 2),
                              round(current_price + 0.5*atr, 2),
                              round(current_price + 1.5*atr, 2)),
            'max_profit': "Net premium received",
            'max_loss': "Difference between strikes - premium (per side)",
            'breakeven': "Lower short put - premium / Upper short call + premium",
            'greek_profile': "Neutral delta, positive theta, negative vega"
        },
        'Equilibrium': {
            'strategy': "Butterfly Spread",
            'direction': "Neutral",
            'strikes': lambda: (round(current_price - atr, 2),
                              round(current_price, 2),
                              round(current_price + atr, 2)),
            'max_profit': "Difference between strikes - net debit",
            'max_loss': "Net premium paid",
            'breakeven': "Middle strike ± net debit",
            'greek_profile': "Neutral delta, positive theta, positive vega"
        }
    }

    strategy = market_state_strategies.get(analysis['market_state'], {})
    
    if strategy:
        print(f"\nRecommended Vertical Spread: {strategy['strategy']}")
        print(f"Directional Bias: {strategy['direction']}")
        
        strikes = strategy['strikes']()
        if strategy['strategy'] == "Iron Condor":
            print(f"Strikes (Put Spread): {strikes[0]} / {strikes[1]}")
            print(f"Strikes (Call Spread): {strikes[2]} / {strikes[3]}")
        elif strategy['strategy'] == "Butterfly Spread":
            print(f"Strikes: {strikes[0]} | {strikes[1]} | {strikes[2]}")
        else:
            print(f"Strikes: {strikes[0]} / {strikes[1]}")
        
        print(f"\nRisk/Reward Profile:")
        print(f"Max Profit: {strategy['max_profit']}")
        print(f"Max Loss: {strategy['max_loss']}")
        print(f"Breakeven: {strategy['breakeven']}")
        print(f"Greek Profile: {strategy['greek_profile']}")
        
        # Add volatility-adjusted suggestions
        if analysis['indicators']:
            vol_profile = "High" if atr > current_price * 0.02 else "Low"
            width = round(2*atr if vol_profile == "High" else atr, 2)
            print(f"\nVolatility Adjustment ({vol_profile}):")
            print(f"Suggested Strike Width: {width} points")
    else:
        print("\nNo specific vertical spread recommendation for current market state")

    # Rest of existing analysis remains unchanged
    print("\nAdditional Market Information:")
    print(f"Bullish Fair Value Gap: {'Present' if analysis['levels']['bullish_fvg'] else 'Absent'}")
    print(f"Bearish Fair Value Gap: {'Present' if analysis['levels']['bearish_fvg'] else 'Absent'}")
    print(f"Order Block: {'Present' if analysis['levels']['order_block'] else 'Absent'}")

    # ... (rest of the existing output code remains the same)

    return analysis

# Example usage
if __name__ == "__main__":
    historical_data = pd.read_csv(r"market_analyzer_optimization/analyzer_ready_data.csv")
    timeframe = input("Enter timeframe (e.g., '1H' for 1 hour, '15T' for 15 minutes): ")
    analysis = analyze_crude_oil(historical_data, timeframe)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13004\183608888.py:12: FutureWarning: 't' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled = data.resample(self.timeframe).agg({


Market Analysis:
Current Market State: Premium

Recommended Vertical Spread: Bear Call Spread
Directional Bias: Bearish
Strikes: 6130.0 / 6139.79

Risk/Reward Profile:
Max Profit: Net premium received
Max Loss: Difference between strikes - premium received
Breakeven: Short strike + net premium
Greek Profile: Negative delta, positive theta, negative vega

Volatility Adjustment (Low):
Suggested Strike Width: 9.79 points

Additional Market Information:
Bullish Fair Value Gap: Absent
Bearish Fair Value Gap: Absent
Order Block: Absent
